In [ ]:
import os
import argparse
import json
import torch

import models.fcmae as fcmae



from MODALITIES import *

In [ ]:
parser = argparse.ArgumentParser()
args = parser.parse_args()

# these 4 arguments need to be set manually
args.checpoint_dir = 'ckpt/pt-all_mod_atto_1M_128_uncertainty/checkpoint-199.pth' # the directory where the model checkpoints are saved
args.random_crop = True # ensure that if the dataset image size is 128 x 128, the resulting image after cropping is 112 x 112.
args.random_crop_size = 112 # the size of the crop (either 112 or 56)
args.patch_size = 16 # patch size used when pretraining. Either 16 or 8 (for img sizes 112 and 56 respectively)
args.loss_aggr = 'uncertainty' # the loss aggregation method. Either 'uncertainty' or 'unweighted'
args.use_orig_stem = False # if True the model uses the original stem as in ConvNeXtV2, else it uses the modified MP-MAE stem.
args.mask_ratio = 0.6 # the ratio of the mask used in the mask prediction task. Either 0.6 or 0.8


# define the input and output bands for the dataset
args.inp_modalities = INP_MODALITIES
args.out_modalities = OUT_MODALITIES

args.modalities = args.inp_modalities.copy()
args.modalities.update(args.out_modalities) # args modalities is a dictionary of all the input and output bands.
args.modalities_full = MODALITIES_FULL # this is a dictionary of all the bands in the dataset.

In [ ]:
checkpoint = torch.load(args.checkpoint_dir, map_location='cpu')
model = fcmae.__dict__['convnextv2_atto'](
    mask_ratio=args.mask_ratio,
    decoder_depth=1,
    decoder_embed_dim=512, 
    norm_pix_loss=True,
    patch_size=args.patch_size,
    img_size=args.random_crop_size,
    args=args
)

In [ ]:
msg = model.load_state_dict(checkpoint['model'], strict=False)
print(msg)